In [1]:
import os
import torch
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline

In [2]:
BASE_MODEL = "beomi/gemma-ko-7b"
FINETUNE_MODEL = "./gemma-ko-7b-finetuning-decoder-with-shuffle"

finetune_model = AutoModelForCausalLM.from_pretrained(FINETUNE_MODEL, device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
print(finetune_model.dtype)

torch.float32


In [4]:
test = pd.read_csv('./data/test.csv')

In [5]:
pipe_finetuned = pipeline(task="text-generation", model=finetune_model, tokenizer=tokenizer)

Device set to use cuda:0


In [23]:
restored_reviews = []

for index, row in test.iterrows():
    print(index)
    query = row['input']
    prompt = (r"""<bos><start_of_turn>user
글자 수와 띄어쓰기를 유지하며 리뷰를 복원하세요:

{}<end_of_turn>
<start_of_turn>model
""".format(query))

    # 텍스트 생성
    generated = pipe_finetuned(
        prompt,
        num_return_sequences=1,
        #temperature=0.2,
        #top_p=0.9,
        max_new_tokens=len(query) + 2,
        do_sample=False,
        eos_token_id=tokenizer.eos_token_id
    )
    
    # 생성된 텍스트에서 출력 부분 추출
    generated_text = generated[0]['generated_text']
    result = generated_text[len(prompt):].strip()
    
    restored_reviews.append(result)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 

In [9]:
def safe_split_newline(x, index):
    parts = x.split("\n")
    if len(parts) > 1:
        return parts[1]
    else:
        print(f"Error at index {index}: {x}")  # 에러가 발생한 인덱스와 값 출력
        return x  # 원래 값 유지 (또는 적절한 처리)

for i in range(len(restored_reviews)):
    safe_split_newline(restored_reviews[i], i)

Error at index 33: 신라스테이 서부산 오픈 축하드립니다!!~<end_of_turn>
Error at index 35: 절대! 다시는 가고 싶지 않음!<end_of_turn>
Error at index 37: 뷰가 정말 예뻤어요!!<end_of_
Error at index 41: 뷰도 청결도 최고였습니다!!!<end_of_turn>
Error at index 42: 직원분들이 친절해서 좋았음.<end_of_turn>
Error at index 52: 포근하니 쾌적하구 룸 컨디션 맘에 쏘옥~<end
Error at index 56: 뷰 맛집... 내 집이면 좋겠어요..ㅜㅜ<end_of_turn>
Error at index 72: 좋아요. 여럿이 놀러 가면 딱이에요~<end_of_
Error at index 73: 다시 가고 싶은 호텔 위치도 최고<end_of_turn>
Error at index 76: 스테이크 먹었는데 맛이 최고예요. 디저트도 굿<end_of_turn>
Error at index 80: 접근성도 좋고, 깨끗하네요.<end_of
Error at index 89: 공용 사우나에 겸미주릴 개 많아요..<end_of_turn>
Error at index 94: 조명이 너무 어두워요.<end_of_
Error at index 99: 32층 다이닝룸 야경이 너무 예뻐요. 로맨틱한 저녁식사<end_of_turn>
Error at index 102: 그다지 권해주고 싶지 않은 곳~<end_of_turn
Error at index 116: 바다눈 좋으나 간격이 너무 좁음.<end_of_turn
Error at index 121: 별내의 호텔 더 메이 의 구청만 내세우는 태도와 고객 불편사항에 대한 불성실, 아고다 측의 환불 규정에 문제가 많다. 당일 숙박을 위한 당일 예약 시 예상 체크인 시간 이전에는 취소가 안 된다 할지라도 숙박 내용 변경은 가능하도록 사용자 입장에서 편리성을 부여했으면 좋겠다. 예약 시에 객실 2개를 예약했다가 업무 일정 변경

In [ ]:
submission = pd.read_csv('./data/sample_submission.csv', encoding = 'utf-8-sig')

submission.to_csv('./submission/augmentation(shuffle)_sampling_submission.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
submission['output'] = restored_reviews
submission['output'] = submission['output'].apply(lambda x: x.split("<end_of_turn>")[0])
submission['output'] = submission['output'].apply(lambda x: x.split("<end")[0])
submission['output'] = submission['output'].fillna('0')

submission.to_csv('./submission/augmentation(shuffle)_sampling_submission.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
for i in range(len(test)):
    test_text = test['input'][i]
    sub_text = submission['output'][i]
    state = True
    
    # 먼저 길이를 맞추기 위해 단어 수와 띄어쓰기 조정
    while len(sub_text) != len(test_text):
        if not state:
            break
        for idx in range(max(test_text, sub_text)):
            if idx == len(test_text):
                state = False
                break
            if (t == ' ') and (s != ' '):
                sub_text = sub_text[:idx] + sub_text[idx+1:]
            elif (t != ' ') and (s == ' '):
                sub_text = sub_text[:idx] + '서' + sub_text[idx:]
                
            break  # 수정했으므로 다시 검사

    # 만약 여전히 길이가 다르면, test 길이에 맞게 자르기
    submission['output'][i] = sub_text[:len(test_text)]

submission.to_csv('./submission/augmentation(shuffle)_sampling_submission.csv', index = False, encoding = 'utf-8-sig')